In [1]:
# Base Data Science snippet
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import time
from tqdm import tqdm_notebook

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../")

import westworld
from westworld.assets.sprites.utils import *
from westworld.assets import make_arrow
from westworld.colors import *
from westworld.environment import GridEnvironment
from westworld.agents.sir_agent import SIRAgent
from westworld.environment.spatial import SpatialEnvironment
from westworld.agents import BaseGridAgent
from westworld.agents.collectible_finder import CollectibleFinderAgent
from westworld.objects import BaseObstacle,BaseTrigger,BaseCollectible,BaseRectangle
from westworld.simulation import Simulation
from westworld.colors import *
from westworld.logger import Logger

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html


# Vis with 3js